In [ ]:
import speech_recognition as sr
import tkinter as tk
from tkinter import messagebox, scrolledtext
import threading

global listening
listening = False

def speech_to_text():
    global listening
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        status_label.config(text="Listening...")
        recognizer.adjust_for_ambient_noise(source)
        
        while listening:
            try:
                audio = recognizer.listen(source, timeout=5)
                text = recognizer.recognize_google(audio, language='en-US')
                print("You said:", text)
                save_to_file(text)
                update_text_display(text)
            except sr.UnknownValueError:
                update_text_display("Could not understand the audio.")
            except sr.RequestError:
                update_text_display("Error: Check internet connection.")
            except sr.WaitTimeoutError:
                update_text_display("Listening timed out...")
    
    status_label.config(text="Click 'Start' to speak")

def save_to_file(text):
    with open("transcription.txt", "a") as file:
        file.write(text + "\n")

def update_text_display(text):
    text_display.insert(tk.END, text + "\n")
    text_display.see(tk.END)

def start_recognition():
    global listening
    listening = True
    threading.Thread(target=speech_to_text, daemon=True).start()
    status_label.config(text="Listening... Speak now")

def stop_recognition():
    global listening
    listening = False
    status_label.config(text="Stopped listening")

# GUI Setup
root = tk.Tk()
root.title("Voice to Text")
root.geometry("500x350")
root.configure(bg="#f4f4f4")

status_label = tk.Label(root, text="Click 'Start' to speak", font=("Arial", 12, "bold"), bg="#e0f7fa", fg="#00796b")
status_label.pack(pady=10)

button_frame = tk.Frame(root, bg="#e0f7fa")
button_frame.pack(pady=10)

button_start = tk.Button(button_frame, text="🎤 Start Recording", command=start_recognition, font=("Arial", 12, "bold"), bg="#4CAF50", fg="white", padx=10, pady=5, relief=tk.RAISED, borderwidth=3)
button_start.grid(row=0, column=0, padx=10)

button_stop = tk.Button(button_frame, text="🛑 Stop Recording", command=stop_recognition, font=("Arial", 12, "bold"), bg="#F44336", fg="white", padx=10, pady=5, relief=tk.RAISED, borderwidth=3)
button_stop.grid(row=0, column=1, padx=10)

text_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=60, height=10, font=("Arial", 10), bg="#ffffff", fg="#000000", borderwidth=3, relief=tk.SUNKEN)
text_display.pack(padx=10, pady=10)

root.mainloop()